[![Generic badge](https://img.shields.io/badge/License-Properitary-red.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/blob/master/LICENSE.md) <br>
[![Active Development](https://img.shields.io/badge/Maintenance%20Level-Actively%20Developed-brightgreen.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/) <br>
[![Ask Me Anything !](https://img.shields.io/badge/Ask%20me-anything-1abc9c.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/issues) <br>
[![Python 3.7.7](https://img.shields.io/badge/python-3.7.7-blue.svg)](https://www.python.org/downloads/release/python-377/)

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logo_fh_hof.svg/2000px-Logo_fh_hof.svg.png" width="300" style="background-color:#FFF">

<font size="+4"><i><u>Q-Learning mit 'Super Mario Bros'</u></i></font>

Seminararbeit der Vorlesung **Angewandtes Maschinelles Lernen** an der **Hochschule für angewande Wissenschaften Hof** des **Sommersemesters 2020**.

# Vorbereitungen

*Code der das Jupyter-Notebook vorbereit.*

## Notebook-Styling

*Ändert u.a. das Layout des Notebooks.*

In [1]:
from IPython.core.display import display, HTML

_notebook_relative_width = "80%" # Wertebereich: 0-100, muss nach HTML-Schema definiert sein
display(HTML('<style>.container { width: ' + _notebook_relative_width + ' !important; }</style>'))

## Requirements

*Die benötigten Packages wurden aus der bei mir installierten Pip-Liste gebildet.*

**Hinweis:** *Es werden* ***mehr als zwingend notwendige*** *Packages installiert.*

In [2]:
# Package-Liste
#!pip list

In [3]:
%%capture
# Installation
#!pip install -r requirements.txt

In [4]:
%%capture
# Update 
#!pip list --outdated --format=freeze | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U

In [5]:
%%capture
# Export Pip-Packages nach './requirements.txt'
!pip freeze > requirements.txt

# Konfiguration

*Import-Statements & Vorbereitung der Frameworks/Module zusammengefasst.*

## Allgemein

*Nicht weiter spezifizierte Import-Statements, bspw. von System-Bibliotheken.*

In [6]:
import os

from datetime import datetime

import numpy as np

import random

In [7]:
# Erstelle Logs-Folder falls nicht existent.
logs_base_dir = "./logs"
if not os.path.isdir(logs_base_dir):
    !mkdir logs

## Tensorflow

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1280px-TensorFlowLogo.svg.png" width="150px"/>

*TensorFlow ist ein Framework zur datenstromorientierten Programmierung. Es wird aus Python-Programmen heraus benutzt und ist in Python und C++ implementiert. Populäre Anwendung findet TensorFlow im Bereich des maschinellen Lernens. [...]*

[Weiter Informationen](https://pypi.org/project/tensorflow/)

In [8]:
%%capture
import tensorflow as tf

## Tensorboard

<img src="https://www.tensorflow.org/site-assets/images/project-logos/tensorboard-logo-social.png" width="250px"/>

*TensorBoard ist ein Sammlung von Web-Applicationen um TensorFlow zu inspizieren und zu verstehen.*

[Weitere Informationen](https://pypi.org/project/tensorboard/)

In [9]:
%load_ext tensorboard

In [10]:
%%capture
from tensorboard import notebook
# Starten einer Tensorboard-Instanz mit Logs & gebindeten Ports
%tensorboard --port=6007 --logdir {logs_base_dir} --bind_all
# Auflisten der TensorBoard-Instanzen (resourcen intesiv)
notebook.list()

## IPyWidgets

In [11]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## PyVirtualDisplay
*Ein python wrapper für Xvfb, Xephyr und Xvnc.*

[Weiter Informationen](https://pypi.org/project/PyVirtualDisplay/)

In [12]:
# videocode dependecies; x11-utils als workaround für 'xdpyinfo'
print("In case of failure in the next Code-Block consider installing python-opengl, xvfb, ffmpeg, x11-utils - Packages via apt-get.\nUse command 'sudo apt-get install -y python-opengl xvfb ffmpeg x11-utils > /dev/null' inside a terminal.")

In case of failure in the next Code-Block consider installing python-opengl, xvfb, ffmpeg, x11-utils - Packages via apt-get.
Use command 'sudo apt-get install -y python-opengl xvfb ffmpeg x11-utils > /dev/null' inside a terminal.


In [13]:
from pyvirtualdisplay import Display as PyVDisplay

global record_base_dir

record_base_dir = "./mp4"
if not os.path.isdir(record_base_dir):
    os.mkdir(record_base_dir)

py_vdisplay = PyVDisplay(visible=0, size=(1400, 900))
py_vdisplay.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1285'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1285'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

## OpenAI-Gym

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/OpenAI_Logo.svg/1920px-OpenAI_Logo.svg.png" width="275px"/>

*OpenAI-Gym ist ein Toolkit für das Entwickeln und Vergleichen von Reinforcement-Learning-Algorithmen.*

[Weitere Informationen](https://pypi.org/project/gym/)

In [14]:
import gym

from gym.wrappers import Monitor
from gym import logger as gymlogger

# DEBUG = 10 # INFO = 20 # WARN = 30 # ERROR = 40 # DISABLED = 50
gymlogger.set_level(50)

## Gym-Super-Mario-Bros

*Eine OpenAI-Gym-Umgebung für Super Mario Bros. und Super Mario Bros. 2 (Lost Levels) auf den NES (Nitendo Entertainment System) mithilfe des nes-py emulators.*

[Weitere Informationen](https://pypi.org/project/gym-super-mario-bros/)

<table><thead><tr><th>SuperMarioBros-v0</th><th>SuperMarioBros-v1</th><th>SuperMarioBros-v2</th><th>SuperMarioBros-v3</th><th>SuperMarioBros2-v0</th><th>SuperMarioBros2-v1</th></tr></thead><tbody><tr><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/c4717c633d3823dda390ebc21bac34b18e7c22c3/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832302d33643135653563322d363833302d313165382d383164342d6563666166666565306131342e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/e8eda56caeefcada9af67f43385ef1f48d0ac394/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831392d33636666366334382d363833302d313165382d383337332d3866616431363635616337322e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7f53e40eb716be49673cf41fb833486ab3ee104a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831382d33636561303964342d363833302d313165382d386566612d3866333464386230356231312e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/51975e7cc634efb02ed92acfb56368733b25f4d9/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831372d33636436363030612d363833302d313165382d386162622d3963656536613331643337372e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/0618011a5c6cedb9dba051b8cf134ba51dd0777a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832322d33643362383431322d363833302d313165382d383630622d6166333830326635333733662e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7c42437f4d2f447e192c088eab22739534c2d9be/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832312d33643264363161322d363833302d313165382d383738392d6139326537353061613961382e706e67" width="175"></td></tr></tbody></table>

In [15]:
import gym_super_mario_bros

from nes_py.wrappers import JoypadSpace, BinarySpaceToDiscreteSpaceEnv # nes_py is a direct dependency by gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY, SIMPLE_MOVEMENT, COMPLEX_MOVEMENT

global gym_enviorment_id
global gym_enviorment_movementset

gym_enviorment_id = 'SuperMarioBros-v0'
gym_enviorment_movementset = 'Simple'

@interact
def show_env_param(Enviorment=['SuperMarioBros-v0', 'SuperMarioBros-v1', 'SuperMarioBros-v2', 'SuperMarioBros-v3', 'SuperMarioBros2-v0', 'SuperMarioBros2-v1', 'SuperMarioBros2-v2', 'SuperMarioBros2-v3'],
                   Movementset=['Simple', 'Complex', 'Only-Right']):
    global gym_enviorment_id
    global gym_enviorment_movementset
    
    gym_enviorment_id = Enviorment
    gym_enviorment_movementset = Movementset
    
    return

interactive(children=(Dropdown(description='Enviorment', options=('SuperMarioBros-v0', 'SuperMarioBros-v1', 'S…

## Stable-Baselines

*Stable Baselines ist eine Sammlung von optimierten Implementation von Reinforcement-Learning-Algorithmen basierend auf OpenAi.*

[Weitere Informationen](https://pypi.org/project/stable-baselines/)

In [16]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2

# Pre-Training

*Code der vor dem Traniern ausgeführt wird.*

## Trainings-Parameter

### Trainings-Epochen

In [17]:
print('Geben Sie eine Anzahl an zu trainierenden Epochen ein (Wertebereich: 1-∞; Default: 1).')

max_amount_of_epoches = 1

@interact
def show_articles_more_than(Epochen=''):
    global max_amount_of_epoches
    try:
        tmp = int(Epochen)
        
        if tmp > 0:
            max_amount_of_epoches = tmp
            print('Epochen-Anzahl in Höhe von {} angewandt.'.format(max_amount_of_epoches))
        else:
            max_amount_of_epoches = 1
            print('Default-Anzahl in Höhe von 1 Epochen angewandt.')
    except ValueError:
        max_amount_of_epoches = 1
        print('Default-Anzahl in Höhe von 1 Epochen angewandt.')

Geben Sie eine Anzahl an zu trainierenden Epochen ein (Wertebereich: 1-∞; Default: 1).


interactive(children=(Text(value='', description='Epochen'), Output()), _dom_classes=('widget-interact',))

### Monitor-Funktion

In [18]:
global gym_enviorment_monitor

gym_enviorment_monitor = True

@interact
def show_monitor_param(Mointor_Run = True):
    global gym_enviorment_monitor
    gym_enviorment_monitor = Mointor_Run
    return

interactive(children=(Checkbox(value=True, description='Mointor_Run'), Output()), _dom_classes=('widget-intera…

## Reward-Funktionen

*Belohnungsfunktionen zum lernen.*

In [19]:
print('test')

test


# Enviorment

In [21]:
"""Ein Wrapper-Objekt für das Super-Mario-Bros-Gym-Enviorment"""
class SuperMarioBrosEnviorment:
    
    ##   Info-Key's   ##
    coins = 'coins'               # Anzahl an eingesammelten Münzen
    flag_get = 'flag_get'         # Ob Mario die Flagge erreicht hat oder ax:
    life = 'life'                 # Marios Anzahl an Leben {3,2,1}
    score = 'score'               # Kumulativer in-game Punktestand
    stage = 'stage'               # Aktuelle Stage {1,2,3,4}
    status = 'status'             # Marios aktueller Status {'small', 'tall', 'fireball'}}
    time = 'time'                 # Die Zeit die übrig ist für den Run
    world = 'world'               # Die Aktuelle Welt {1, 2, 3, 4, 5, 6, 7, 8}
    x_pos = 'x_pos'               # Marios absolute X-Position (von links ausgehend)
    x_pos_screen = 'x_pos_screen' # Marios relative X-Position (von links ausgehend)
    y_pos = 'y_pos'               # Marios relative Y-Postion (von unten ausgehend)
    
    # Init-Funktion
    def __init__(self):
        global gym_enviorment_id
        global gym_enviorment_movementset
        
        env = gym.make(gym_enviorment_id)
        
        if gym_enviorment_movementset == 'Simple':
             self.env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)
        elif gym_enviorment_movementset == 'Complex':
             self.env = BinarySpaceToDiscreteSpaceEnv(env, COMPLEX_MOVEMENT)
        elif gym_enviorment_movementset == 'Only-Right':
             self.env = BinarySpaceToDiscreteSpaceEnv(env, RIGHT_ONLY)
        else:
            raise Error('Invalid Movementset. Possible Movementset are \'SIMPLE_MOVEMENT\', \'COMPLEX_MOVEMENT\' or \'RIGHT_ONLY\'.')
    
    # Ändert das Enviorment basierend auf den Aktionen des Agents
    def step(self, action):
        next_state, _, done, info = self.env.step(action)
        reward = self.reward(info)
        return next_state, reward, done, info
    
    # Setzt das Enviorment auf den initalen State
    def reset(self):
        return self.env.reset()
    
    # Zeigt den neuen State des Enviorments auf einen Screen
    def render(self):
        return self.env.render()
    
    # Berechnet die Reward-Funktion
    def reward(info):
        # Variablen aus dem Info-Dict lesen
        coins = info[coins]
        flag_get = info[flag_get]
        life = info[life]
        score = info[score]
        stage = info[stage]
        status = info[status]
        time = info[time]
        world = info[world]
        x_pos = info[x_pos]
        x_pos_screen = info[x_pos_screen]
        y_pos = info[y_pos]
        
        return 0

# Agent

In [ ]:
class SuperMarioBrosAgent:
    def __init__(self):
        pass

    def action(self, state):
        if state > 0:
            return 1
        else:
            return 0

# Hyperparameter

def init_model(gym_enviorment):
    global model
    
    model = PPO2(MlpPolicy, gym_enviorment, verbose = 1, tensorboard_log=logs_base_dir)
    #model.learn(totald_timesteps=25000)

In [ ]:
total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 100              # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [ ]:
def init_qtable(gym_enviorment):
    action_size = gym_enviorment.action_space.n
    state_size = gym_enviorment.observation_space.n

    qtable = np.zeros((state_size, action_size))